<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/Greek/sub/sub1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Greek Language


In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
!pip install focal-loss
!pip install keras-radam

In [3]:
cd /content/drive/My Drive/dataset/OffenseEval2020/data/Greek/

/content/drive/My Drive/dataset/OffenseEval2020/data/Greek


In [4]:
ls

cc.el.300.vec     offenseval-greek-training-v1.tsv  testset_taska.tsv
cc.el.300.vec.gz  readme-trainingset-greek.txt


In [0]:
import pandas as pd
train=pd.read_csv( 'offenseval-greek-training-v1.tsv',sep="\t")
train["tweet"]= train["tweet"].str.replace('@USER', "") 
train['tweet']=train['tweet'].str.replace('\d+', '')
train['tweet']=train['tweet'].str.replace('URL', '')
train["tweet"]= train["tweet"].str.lower()
train=train.sample(frac=1)

In [6]:
train.head()

,id,tweet,subtask_a
3774,8353,από μένα είναι στο μουνι μου και πάμε παρακάτω,OFF
625,8325,"""η γιαγιά μας η καλή"" το όνομα του επεισοδίου ...",NOT
4779,1274,συγκινήσου μωρέ κωστή λιγακι. #gynaikaxwrisonoma,NOT
7159,9305,καριέρα στο εξωτερικό η μαρτίνα; πού στο τσίρκ...,NOT
4281,3740,να είσαι πάντα καλά! το όνομα μου όντως κωνστ...,NOT


In [7]:
train.tail()

,id,tweet,subtask_a
2252,6585,το να πεις αυτο το τραγουδι με δικο σου τροπο ...,NOT
4965,4109,πιπιλα εισαι,NOT
6216,3847,sorry κι όλας αλλά παντελής έλλειψη γούστου απ...,NOT
953,7309,"άιντε σουζάνα, φτιάξε βαλίτσες ! #gntmgr",NOT
1808,890,άντε γαμήσου έπρεπε να του πεις,OFF


In [0]:
labels=train['subtask_a']
train=train['tweet']
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
labels=le.fit_transform(labels)

In [9]:
import collections
collections.Counter(labels)

Counter({0: 6257, 1: 2486})

In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 200

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(train)
sequences = tokenize.texts_to_sequences(train)
word_index = tokenize.word_index
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

Using TensorFlow backend.


In [11]:
len(sequences_matrix),len(labels)

(8743, 8743)

In [0]:
from keras.models import Sequential
from keras.layers import Embedding,CuDNNGRU,GRU,Dense,Dropout,Bidirectional,GlobalMaxPool1D,GlobalAveragePooling1D, SpatialDropout1D
from keras.optimizers import RMSprop
from keras.initializers import Constant
from sklearn.utils import class_weight
from sklearn.metrics import classification_report,f1_score
import keras
from keras_radam import RAdam
import numpy as np
from sklearn.model_selection import StratifiedKFold
from focal_loss import BinaryFocalLoss
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [13]:
num_words = min(max_words, len(word_index)) + 1
print(num_words)

10001


In [0]:
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.el.300.vec.gz

In [0]:
# import gzip
# import shutil
# with gzip.open('cc.el.300.vec.gz', 'rb') as f_in:
#     with open('cc.el.300.vec', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

In [0]:
embed_size=300
def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')

def build_matrix(embedding_path, word_index):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

    nb_words = min(max_words, len(word_index))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    for word, i in word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [0]:
embeddings=build_matrix('cc.el.300.vec', word_index)


In [19]:
embeddings.shape

(10001, 300)

In [0]:
def model_func():
  model = Sequential()
  model.add(Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False))
  model.add(Bidirectional(CuDNNGRU(150, return_sequences = True)))
  model.add((CuDNNGRU(32, return_sequences = True)))
  model.add(GlobalMaxPool1D())
  model.add(Dense(100, activation="relu"))
  model.add(Dense(50, activation="relu"))
  #model.add(Dense(10, activation="relu"))

  model.add(Dropout(0.05))
  model.add(Dense(1, activation="sigmoid"))
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer='adam', metrics=['accuracy'])
  return model

In [0]:
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',np.unique(labels),labels)
class_weights=dict(enumerate(class_weights))

In [0]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1)

In [23]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(sequences_matrix,labels,test_size=0.15,stratify=labels,shuffle=True)
model=model_func()
model.fit(x_train,y_train,batch_size=64,epochs=10,verbose=2,class_weight=class_weights,validation_data=(x_val,y_val))

Train on 7431 samples, validate on 1312 samples
Epoch 1/10
 - 20s - loss: 0.1458 - acc: 0.7165 - val_loss: 0.1209 - val_acc: 0.7812
Epoch 2/10
 - 6s - loss: 0.1140 - acc: 0.8128 - val_loss: 0.1102 - val_acc: 0.8270
Epoch 3/10
 - 6s - loss: 0.1043 - acc: 0.8408 - val_loss: 0.1082 - val_acc: 0.8323
Epoch 4/10
 - 6s - loss: 0.1006 - acc: 0.8466 - val_loss: 0.1062 - val_acc: 0.8415
Epoch 5/10
 - 6s - loss: 0.0960 - acc: 0.8584 - val_loss: 0.1090 - val_acc: 0.8377
Epoch 6/10
 - 6s - loss: 0.0919 - acc: 0.8583 - val_loss: 0.1083 - val_acc: 0.8270
Epoch 7/10
 - 6s - loss: 0.0861 - acc: 0.8681 - val_loss: 0.1175 - val_acc: 0.8209
Epoch 8/10
 - 6s - loss: 0.0798 - acc: 0.8812 - val_loss: 0.1232 - val_acc: 0.8148
Epoch 9/10
 - 6s - loss: 0.0733 - acc: 0.8922 - val_loss: 0.1259 - val_acc: 0.8171
Epoch 10/10
 - 6s - loss: 0.0649 - acc: 0.9074 - val_loss: 0.1517 - val_acc: 0.8140


In [30]:
from sklearn.metrics import classification_report

y_pred = model.predict(x_val, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred = (y_pred > 0.5)

print(classification_report(y_val, y_pred))

1312/1312 [==============================] - 0s 252us/step
              precision    recall  f1-score   support

           0       0.85      0.90      0.87       939
           1       0.71      0.59      0.64       373

    accuracy                           0.81      1312
   macro avg       0.78      0.75      0.76      1312
weighted avg       0.81      0.81      0.81      1312



In [31]:
test=pd.read_csv( 'testset_taska.tsv',sep='\t')
test["tweet"]= test["tweet"].str.replace('@USER', "") 
test['tweet']=test['tweet'].str.replace('\d+', '')
test['tweet']=test['tweet'].str.replace('URL', '')
test['tweet']=test['tweet'].str.lower()

test.head()

,id,tweet
0,2707,θέλω να των δω από εδώ και εμπρός αν δεν κάνε...
1,2251,#survivorgr α και φορές και με διαφορετικούς....
2,9814,και μου έλεγε η γυναίκα μου το πρωί πάρε την τ...
3,8949,κατω τα χερια απο τον #κυρανακης
4,6913,"μην μας το παιζεις πονοψυχη,κρυφορατσιστρια τ..."


In [0]:
X_test=test['tweet']

test_sequences = tokenize.texts_to_sequences(X_test)
X_test_sequences = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [33]:
y_test = model.predict(X_test_sequences, batch_size=64, verbose=1)
y_test[y_test>0.5]=1
y_test[y_test<0.5]=0
y_test=y_test.astype('int16').ravel()

1544/1544 [==============================] - 0s 248us/step


In [34]:
y_test=le.inverse_transform(y_test)
y_test=pd.DataFrame(y_test,columns=['label'])
y_test=pd.concat([test['id'], y_test['label']], axis=1)
y_test.head()

,id,label
0,2707,NOT
1,2251,NOT
2,9814,NOT
3,8949,NOT
4,6913,NOT


In [0]:
y_test.to_csv('/content/sub1.csv',index=False,header=None)


In [0]:
import keras
keras.backend.clear_session()